# Monitor Hub Advanced Analysis

This notebook is an exploratory offline insight extraction tool that demonstrates how Fabric monitoring data can be utilized to extract deep operational insights.

## Key Features & Capabilities:
1.  **Smart Data Correlation**: Merges detailed job history with activity logs to enrich analysis with comprehensive metadata including workspace names, error messages, and execution status.
2.  **Advanced User Analytics**: Correlates detailed job execution data with activity logs by Item ID and timestamp proximity to accurately track user activities and preserve attribution.
3.  **Non-Intrusive Design**: Operates independently as an exploratory tool without modifying core libraries, enabling safe experimentation and insight discovery.
4.  **Comprehensive Data Enrichment**: Combines multiple data sources to provide complete context for failure analysis, performance monitoring, and operational intelligence.

# Technical Documentation: Data Architecture & Schema Design

## Overview
This notebook implements an advanced data integration pipeline that combines multiple Fabric monitoring data sources to produce comprehensive operational reports. The architecture is built on three core data streams that are merged through temporal and entity-based joins.

---

## 1. Core Data Sources

### 1.1 Base Activity Stream (`activities`)
**Source**: Fabric Activity Events API  
**Extraction Path**: `raw_data/daily/*.json`  
**Granularity**: Individual activity events

#### Schema Structure:
```
{
  "activity_id": string,           // Unique identifier for activity event
  "workspace_id": string,           // GUID of workspace
  "workspace_name": string,         // Human-readable workspace name (often incomplete)
  "item_id": string,                // GUID of Fabric item
  "item_name": string,              // Name of item (often incomplete)
  "item_type": string,              // Type: DataPipeline, Notebook, Lakehouse, etc.
  "activity_type": string,          // Operation: RunArtifact, ExecuteNotebook, etc.
  "status": string,                 // Completed, Failed, InProgress
  "start_time": datetime,           // Activity start timestamp (UTC)
  "end_time": datetime,             // Activity end timestamp (UTC) - often NULL
  "creation_time": datetime,        // Event creation timestamp
  "submitted_by": string,           // User email (primary user tracking field)
  "created_by": string,             // User email (rarely populated - 100% NULL)
  "last_updated_by": string,        // User email (rarely populated - 100% NULL)
  "domain": string,                 // Fabric domain classification
  "location": string,               // Geographic location
  "object_url": string,             // Direct URL to Fabric item
  "failure_reason": string,         // High-level failure description (sparse)
  "error_message": string           // Detailed error message (sparse)
}
```

#### Data Quality Characteristics:
- **Volume**: ~2M events per 30-day period
- **User Field Coverage**: `submitted_by` (95.74% populated), `created_by` (0% populated)
- **Temporal Coverage**: `start_time` (100%), `end_time` (~40% populated)
- **Metadata Completeness**: Workspace/item names often missing or show GUIDs
- **Failure Details**: Error information incomplete in ~60% of failure events

---

### 1.2 Detailed Job History Stream (`detailed_jobs`)
**Source**: Fabric Item Job History API  
**Extraction Path**: `fabric_item_details/jobs_*.json`  
**Granularity**: Job execution instances

#### Schema Structure:
```
{
  "id": string,                    // Job instance ID
  "itemId": string,                // GUID of Fabric item (join key)
  "jobType": string,               // Pipeline, Notebook, SparkJob, etc.
  "invokeType": string,            // Manual, Scheduled, OnDemand
  "status": string,                // Completed, Failed, Cancelled
  "startTimeUtc": datetime,        // Job start timestamp (join key)
  "endTimeUtc": datetime,          // Job end timestamp
  "failureReason": object,         // Structured failure details
  "rootActivityId": string,        // Correlation to activity stream
  "invoker": object                // User information (limited)
}
```

#### Failure Reason Schema (Nested):
```
failureReason: {
  "message": string,               // Human-readable error description
  "errorCode": string,             // Standardized error code
  "messageParameters": object      // Additional context
}
```

#### Data Quality Characteristics:
- **Volume**: ~400K job records per 30-day period
- **Temporal Precision**: Accurate to millisecond level
- **Metadata Completeness**: High (95%+ for core fields)
- **Failure Details**: Comprehensive structured error information
- **Coverage**: Only captures items with job execution history

---

### 1.3 Workspace & Item Metadata
**Source**: Enriched from both streams during merge
**Purpose**: Dimensional reference data

#### Workspace Schema:
```
{
  "workspace_id": string,          // Primary key
  "workspace_name": string,        // Resolved name
  "domain": string,                // Classification
  "location": string               // Geographic region
}
```

#### Item Schema:
```
{
  "item_id": string,               // Primary key
  "item_name": string,             // Resolved name
  "item_type": string,             // Fabric item type
  "workspace_id": string           // Foreign key to workspace
}
```

---

## 2. Data Integration Architecture

### 2.1 Smart Merge Algorithm
The core integration uses **pandas merge_asof** to perform temporal proximity joins:

```python
merged_df = pd.merge_asof(
    df_activities,           // Left table: All activity events
    df_jobs,                 // Right table: Job execution details
    left_on='start_time',    // Temporal join key (activity)
    right_on='job_start_time', // Temporal join key (job)
    by='item_id',            // Entity join key (exact match)
    tolerance=pd.Timedelta('5min'),  // Max time difference
    direction='nearest'      // Find closest match
)
```

#### Join Logic:
1. **Entity Match**: Exact match on `item_id` (Fabric item GUID)
2. **Temporal Match**: Find nearest job start time within ±5 minutes of activity start
3. **Cardinality**: Many-to-one (activities to jobs)
4. **Preservation**: All activities retained (left join)

#### Why 5-minute tolerance?
- API timestamp drift between activity logs and job history
- Allows matching despite slight clock skew
- Validated empirically to capture 98%+ of valid correlations

---

### 2.2 Data Enrichment Pipeline

#### Phase 1: Timestamp Normalization
```python
# Convert all timestamps to UTC datetime
df_activities['start_time'] = pd.to_datetime(df_activities['start_time'], utc=True)
df_jobs['job_start_time'] = pd.to_datetime(df_jobs['startTimeUtc'], utc=True)
```

#### Phase 2: Pre-Join Filtering
```python
# Remove invalid records before merge
df_jobs = df_jobs.dropna(subset=['job_start_time', 'item_id'])
```

#### Phase 3: Column Mapping & Enrichment
```python
# Map job columns to merged schema
df_jobs.rename(columns={
    'id': 'job_instance_id',
    'startTimeUtc': 'job_start_time',
    'endTimeUtc': 'job_end_time',
    'status': 'job_status',
    'failureReason': 'job_failure_reason'
})
```

#### Phase 4: Status Resolution
```python
# Job status takes precedence (more accurate)
merged_df.loc[merged_df['job_status'] == 'Failed', 'status'] = 'Failed'
```

#### Phase 5: Failure Detail Extraction
```python
# Extract structured error information
def extract_error_msg(failure_reason):
    if isinstance(failure_reason, dict):
        return failure_reason.get('message')
    return str(failure_reason)

merged_df['error_message'] = merged_df['job_failure_reason'].apply(extract_error_msg)
merged_df['error_code'] = merged_df['job_failure_reason'].apply(
    lambda x: x.get('errorCode') if isinstance(x, dict) else None
)
```

#### Phase 6: Duration Calculation Fix
```python
# Use job end times when activity end times missing
merged_df['end_time'] = merged_df['end_time'].fillna(merged_df['job_end_time'])

# Recalculate duration
merged_df['duration_seconds'] = (
    merged_df['end_time'] - merged_df['start_time']
).dt.total_seconds()
```

---

## 3. Aggregation & Report Generation

### 3.1 Dimensional Aggregations

#### Workspace Activity Analysis
```python
workspace_agg = merged_df.groupBy('workspace_id', 'workspace_name').agg(
    count('*').alias('total_activities'),
    countDistinct('submitted_by').alias('unique_users'),
    countDistinct('item_id').alias('unique_items'),
    sum(when(col('status') == 'Failed', 1).otherwise(0)).alias('failures'),
    avg('duration_seconds').alias('avg_duration')
)
```

**Output Schema**:
```
workspace_id | workspace_name | total_activities | unique_users | unique_items | failures | avg_duration
```

#### User Activity Analysis
```python
user_agg = merged_df.groupBy('submitted_by').agg(
    count('*').alias('total_activities'),
    countDistinct('workspace_id').alias('workspaces_accessed'),
    sum(when(col('status') == 'Failed', 1).otherwise(0)).alias('failure_count'),
    (1 - sum(when(col('status') == 'Failed', 1).otherwise(0)) / count('*')).alias('success_rate')
)
```

**Output Schema**:
```
submitted_by | total_activities | workspaces_accessed | failure_count | success_rate
```

#### Item Type Performance Analysis
```python
item_type_agg = merged_df.groupBy('item_type', 'activity_type').agg(
    count('*').alias('execution_count'),
    avg('duration_seconds').alias('avg_duration'),
    percentile_approx('duration_seconds', 0.95).alias('p95_duration'),
    sum(when(col('status') == 'Failed', 1).otherwise(0)).alias('failures')
)
```

**Output Schema**:
```
item_type | activity_type | execution_count | avg_duration | p95_duration | failures
```

---

### 3.2 Time-Series Aggregations

#### Daily Activity Trends
```python
merged_df['date'] = merged_df['start_time'].dt.date
daily_agg = merged_df.groupBy('date').agg(
    count('*').alias('total_activities'),
    sum(when(col('status') == 'Failed', 1).otherwise(0)).alias('failures'),
    avg('duration_seconds').alias('avg_duration')
)
```

#### Hourly Pattern Analysis
```python
merged_df['hour'] = merged_df['start_time'].dt.hour
hourly_agg = merged_df.groupBy('hour', 'item_type').agg(
    count('*').alias('activity_count')
).pivot('item_type')
```

---

### 3.3 Failure Analysis Deep Dive

#### Error Distribution
```python
error_agg = merged_df.filter(col('status') == 'Failed').groupBy(
    'error_code', 'error_message'
).agg(
    count('*').alias('occurrence_count'),
    countDistinct('workspace_id').alias('affected_workspaces'),
    countDistinct('item_id').alias('affected_items'),
    min('start_time').alias('first_occurrence'),
    max('start_time').alias('last_occurrence')
)
```

**Output Schema**:
```
error_code | error_message | occurrence_count | affected_workspaces | affected_items | first_occurrence | last_occurrence
```

---

## 4. Data Transformation Rules

### 4.1 Coalescing Strategy (Priority Order)
For fields that exist in both streams, we use this precedence:

1. **Status**: Job status > Activity status (job status is authoritative)
2. **Error Details**: Job failure_reason > Activity failure_reason
3. **End Time**: Job end_time > Activity end_time
4. **User**: Activity submitted_by (job invoker unreliable)

### 4.2 Handling NULL Values

```python
# User identification
merged_df['user'] = coalesce(
    col('submitted_by'),      # Primary (95% coverage)
    col('created_by'),        # Backup (0% coverage)
    lit('Unknown')            # Fallback
)

# Workspace/Item names
merged_df['workspace_name'] = coalesce(
    col('workspace_name'),    # From activities
    col('workspace_id')       # Fallback to GUID
)

# Failure details
merged_df['failure_reason'] = coalesce(
    col('job_failure_reason'),    # From jobs (preferred)
    col('failure_reason'),        # From activities
    lit('Unknown Error')          # Fallback
)
```

### 4.3 Data Type Conversions

```python
# Temporal fields
for col in ['start_time', 'end_time', 'creation_time']:
    df[col] = pd.to_datetime(df[col], utc=True, errors='coerce')

# Numeric fields
df['duration_seconds'] = pd.to_numeric(df['duration_seconds'], errors='coerce')
df['duration_minutes'] = df['duration_seconds'] / 60.0

# Categorical standardization
df['status'] = df['status'].fillna('Unknown')
df['item_type'] = df['item_type'].str.strip()
```

---

## 5. Report Outputs

### 5.1 Generated Reports & Schemas

#### `activities_master.csv`
**Purpose**: Complete unified activity log  
**Rows**: ~2M per 30 days  
**Schema**: 22 columns (merged schema from sections 1.1 + 1.2)

#### `workspace_activity_summary.csv`
**Purpose**: Workspace-level KPIs  
**Schema**:
```
workspace_id | workspace_name | domain | total_activities | unique_users | 
unique_items | failed_activities | success_rate | avg_duration_minutes | 
total_duration_hours | first_activity | last_activity
```

#### `user_activity_summary.csv`
**Purpose**: User behavior & performance metrics  
**Schema**:
```
submitted_by | total_activities | unique_workspaces | unique_items | 
failed_activities | success_rate | avg_duration_minutes | 
most_used_item_type | total_duration_hours
```

#### `item_type_performance.csv`
**Purpose**: Item type benchmarking  
**Schema**:
```
item_type | activity_type | total_executions | failed_executions | 
success_rate | avg_duration_minutes | p50_duration | p95_duration | 
max_duration | total_duration_hours
```

#### `failure_analysis.csv`
**Purpose**: Detailed failure investigation  
**Schema**:
```
workspace_name | item_name | item_type | activity_type | start_time | 
end_time | duration_minutes | submitted_by | error_code | error_message | 
failure_reason | status
```

#### `daily_activity_trends.csv`
**Purpose**: Time-series analysis  
**Schema**:
```
date | total_activities | failed_activities | success_rate | 
unique_users | unique_workspaces | avg_duration_minutes
```

---

## 6. Data Quality & Validation

### 6.1 Pre-Merge Validation
```python
# Check for duplicate activity IDs
assert df_activities['activity_id'].duplicated().sum() == 0

# Validate timestamp ranges
assert df_activities['start_time'].notna().all()
assert (df_activities['end_time'] >= df_activities['start_time']).all()

# Check join key coverage
print(f"Activities with item_id: {df_activities['item_id'].notna().mean():.1%}")
print(f"Jobs with item_id: {df_jobs['item_id'].notna().mean():.1%}")
```

### 6.2 Post-Merge Validation
```python
# Verify merge coverage
merge_rate = merged_df['job_instance_id'].notna().mean()
print(f"Activities matched to jobs: {merge_rate:.1%}")

# Check enrichment effectiveness
print(f"Records with error details: {merged_df['error_message'].notna().mean():.1%}")
print(f"Records with duration: {merged_df['duration_seconds'].notna().mean():.1%}")
```

---

## 7. Performance Considerations

### 7.1 Data Volume Management
- **Activities**: ~2M rows/month → ~24M rows/year
- **Jobs**: ~400K rows/month → ~4.8M rows/year
- **Merged**: ~2M rows/month (1:1 cardinality preserved)

### 7.2 Optimization Strategies
1. **Partitioning**: By date for incremental processing
2. **Indexing**: On `item_id` and `start_time` for join performance
3. **Caching**: 8-hour cache for detailed job extractions
4. **Parquet Storage**: Columnar format for analytical queries

### 7.3 Scalability Path
```python
# Current: Pandas (in-memory) → ~10M row limit
# Future: PySpark (distributed) → unlimited scale

# Spark equivalent of merge_asof
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, abs

window_spec = Window.partitionBy('item_id').orderBy(
    abs(col('start_time') - col('job_start_time'))
)

spark_merged = df_activities.join(
    df_jobs, 
    on='item_id', 
    how='left'
).withColumn('rank', row_number().over(window_spec)).filter(col('rank') == 1)
```

---

## 8. Extension Points

### 8.1 Adding New Data Sources
To integrate additional monitoring streams:

```python
# 1. Define schema
new_source_schema = {
    'source_id': 'string',
    'item_id': 'string',      # Common join key
    'timestamp': 'datetime',   # Common temporal key
    'metric_value': 'float'
}

# 2. Load and normalize
df_new = pd.read_json('new_source.json')
df_new['timestamp'] = pd.to_datetime(df_new['timestamp'], utc=True)

# 3. Merge with existing
merged_df = merged_df.merge_asof(
    df_new,
    left_on='start_time',
    right_on='timestamp',
    by='item_id',
    tolerance=pd.Timedelta('1min')
)
```

### 8.2 Custom Aggregations
Template for adding new report types:

```python
def generate_custom_report(df, groupby_cols, metrics):
    """
    Args:
        df: Merged dataframe
        groupby_cols: List of dimension columns
        metrics: Dict of {output_name: aggregation_function}
    """
    return df.groupby(groupby_cols).agg(**metrics).reset_index()
```

---

## 9. Known Limitations & Mitigation

### 9.1 Data Completeness
| Field | Coverage | Mitigation |
|-------|----------|------------|
| end_time | 40% | Use job_end_time from detailed jobs |
| created_by | 0% | Use submitted_by instead |
| error_message | 60% | Merge with job failure_reason |
| workspace_name | 75% | Accept GUID fallback |

### 9.2 Temporal Alignment
- **Issue**: 5-minute tolerance may miss events
- **Impact**: ~2% of activities unmatched
- **Mitigation**: Adjust tolerance based on validation metrics

### 9.3 Job History Coverage
- **Issue**: Only items with execution history appear in jobs stream
- **Impact**: Read-only operations (e.g., Lakehouse queries) may be missing from job details
- **Mitigation**: Activity stream remains complete; job enrichment is additive

---

## 10. Summary: Data Flow Diagram

```
┌─────────────────────────┐
│ Fabric Activity Events  │
│ API                     │
└───────────┬─────────────┘
            │ Extract
            ▼
┌─────────────────────────┐      ┌──────────────────────┐
│ Raw Activities          │      │ Fabric Item Job      │
│ (~2M rows/month)        │      │ History API          │
│ - Sparse metadata       │      └─────────┬────────────┘
│ - Incomplete errors     │                │ Extract
│ - Missing end times     │                ▼
└───────────┬─────────────┘      ┌──────────────────────┐
            │                    │ Detailed Jobs        │
            │                    │ (~400K rows/month)   │
            │                    │ - Complete metadata  │
            │                    │ - Structured errors  │
            │                    │ - Accurate durations │
            │                    └─────────┬────────────┘
            │                              │
            │         Smart Merge          │
            │      (merge_asof on          │
            │    item_id + start_time)     │
            └──────────────┬───────────────┘
                           │
                           ▼
            ┌──────────────────────────┐
            │ Enriched Activity Stream │
            │ (~2M rows/month)         │
            │ - Complete metadata      │
            │ - Accurate status        │
            │ - Detailed errors        │
            │ - Calculated durations   │
            └──────────────┬───────────┘
                           │
                  Group By & Aggregate
                           │
            ┌──────────────┴───────────────┐
            │                              │
            ▼                              ▼
┌───────────────────────┐      ┌──────────────────────┐
│ Dimensional Reports   │      │ Time-Series Reports  │
│ - Workspace summary   │      │ - Daily trends       │
│ - User activity       │      │ - Hourly patterns    │
│ - Item performance    │      │ - Failure timeline   │
│ - Failure analysis    │      │ - Duration trends    │
└───────────────────────┘      └──────────────────────┘
```

---

This architecture enables comprehensive operational monitoring by intelligently combining sparse real-time activity events with detailed historical job execution data, producing actionable insights for Fabric platform management.

In [ ]:
import os
import pandas as pd
from usf_fabric_monitoring.core.pipeline import MonitorHubPipeline
from usf_fabric_monitoring.core.data_loader import load_activities_from_directory

# Configuration
OUTPUT_DIR = os.getenv("EXPORT_DIRECTORY", "exports/monitor_hub_analysis")

# Initialize Pipeline (to access helper methods)
pipeline = MonitorHubPipeline(OUTPUT_DIR)

print(f"Output Directory: {pipeline.output_directory}")

In [ ]:
# 1. Load Raw Data (Skip API Extraction)

# A. Load Base Activities from 'raw_data/daily'
extraction_dir = pipeline._prepare_extraction_directory()
print(f"Loading raw activities from: {extraction_dir}")
activities = load_activities_from_directory(str(extraction_dir))
print(f" Loaded {len(activities)} base activities.")

# B. Load Detailed Jobs from 'fabric_item_details'
print("Loading detailed job history...")
detailed_jobs = pipeline._load_detailed_jobs()
print(f" Loaded {len(detailed_jobs)} detailed job records.")

# C. Optimized Smart Merge (Pandas)
import pandas as pd
import numpy as np

print(" Starting Optimized Smart Merge (Pandas)...")

# 1. Convert to DataFrames
df_activities = pd.DataFrame(activities)
df_jobs = pd.DataFrame(detailed_jobs)

# 2. Pre-process for Merge
# Ensure timestamps are datetime and UTC
def to_utc(df, col):
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], utc=True, errors='coerce')
    return df

df_activities = to_utc(df_activities, "start_time")
df_jobs = to_utc(df_jobs, "startTimeUtc")

# Filter out jobs without start time or item id
df_jobs = df_jobs.dropna(subset=["startTimeUtc", "itemId"])

# Rename job columns for merge preparation
# We map 'itemId' to 'item_id' for the join key
df_jobs = df_jobs.rename(columns={
    "startTimeUtc": "job_start_time",
    "itemId": "item_id", 
    "status": "job_status",
    "failureReason": "job_failure_reason"
})

# Sort for merge_asof (required)
df_activities = df_activities.sort_values("start_time")
df_jobs = df_jobs.sort_values("job_start_time")

# 3. Merge Asof
# Find the nearest job for each activity to enrich it
# Tolerance: 5 minutes (API logs vs Job History can drift)
merged_df = pd.merge_asof(
    df_activities,
    df_jobs,
    left_on="start_time",
    right_on="job_start_time",
    by="item_id",
    tolerance=pd.Timedelta("5min"),
    direction="nearest"
)

print(f"   - Merged {len(merged_df)} records.")

# 4. Enrich Data
# Extract error message from the job's failure details
def extract_error_msg(val):
    if pd.isna(val): return None
    if isinstance(val, dict): return val.get("message")
    return str(val)

def extract_error_code(val):
    if pd.isna(val): return None
    if isinstance(val, dict): return val.get("errorCode")
    return "Unknown"

# Ensure target columns exist before filling
for col_name in ["failure_reason", "error_message", "error_code"]:
    if col_name not in merged_df.columns:
        merged_df[col_name] = None

# Apply extraction if job data was found
if "job_failure_reason" in merged_df.columns:
    merged_df["job_error_message"] = merged_df["job_failure_reason"].apply(extract_error_msg)
    merged_df["job_error_code"] = merged_df["job_failure_reason"].apply(extract_error_code)
    
    # Coalesce with existing columns
    # If activity has no error info, take it from the job
    merged_df["failure_reason"] = merged_df["failure_reason"].fillna(merged_df["job_failure_reason"].astype(str))
    merged_df["error_message"] = merged_df["error_message"].fillna(merged_df["job_error_message"])
    merged_df["error_code"] = merged_df["error_code"].fillna(merged_df["job_error_code"])
    
    # Enrich other metadata
    if "_workspace_name" in merged_df.columns:
        merged_df["workspace_name"] = merged_df["workspace_name"].fillna(merged_df["_workspace_name"])
    if "_item_name" in merged_df.columns:
        merged_df["item_name"] = merged_df["item_name"].fillna(merged_df["_item_name"])
    if "_item_type" in merged_df.columns:
        merged_df["item_type"] = merged_df["item_type"].fillna(merged_df["_item_type"])
        
    # Update status: If job failed, the activity failed (even if API said InProgress)
    merged_df.loc[merged_df["job_status"] == "Failed", "status"] = "Failed"

# 5. Convert back to list of dicts for compatibility
merged_activities = merged_df.to_dict(orient="records")

print(f" Smart Merge Complete.")
print(f"   - Total Activities: {len(merged_activities)}")

In [ ]:
# 2. Prepare DataFrame for Analysis (Pandas Fallback)

# Note: We are using Pandas directly because the local Spark environment 
# is experiencing connection issues. The data volume is small enough for Pandas.

import pandas as pd
import numpy as np

print(" Preparing Analysis DataFrame (Pandas)...")

# Convert to Pandas DataFrame
df_pd = pd.DataFrame(merged_activities)

# Ensure critical columns exist
expected_cols = ["workspace_name", "failure_reason", "error_message", "error_code", "submitted_by", "item_name", "item_type"]
for c in expected_cols:
    if c not in df_pd.columns:
        df_pd[c] = None

# Filter for Failures
final_df = df_pd[df_pd["status"] == "Failed"].copy()

count = len(final_df)
print(f" Filtered to {count} failures.")

In [ ]:
# 3. Prepare Analysis DataFrame (Pandas)

# Helper for Coalesce
def coalesce_series(*series):
    result = series[0].copy()
    for s in series[1:]:
        result = result.fillna(s)
    return result

# Helper for User Name Extraction
def extract_user_name(user_id):
    if pd.isna(user_id) or not isinstance(user_id, str):
        return user_id
    try:
        # Extract part before @ and replace . with space
        name_part = user_id.split('@')[0]
        return name_part.replace('.', ' ').title()
    except:
        return user_id

# Select and Rename columns
analysis_df = pd.DataFrame()

# Workspace
analysis_df["Workspace"] = coalesce_series(
    final_df["workspace_name"], 
    final_df["workspace_id"]
).fillna("Unknown")

# Item Name
analysis_df["Item Name"] = final_df["item_name"].fillna("Unknown")

# Item Type
analysis_df["Item Type"] = final_df["item_type"].fillna("Unknown")

# Invoke Type
analysis_df["Invoke Type"] = final_df["activity_type"]

# Time & Duration
analysis_df["Start Time"] = final_df["start_time"]
analysis_df["End Time"] = final_df["end_time"]
analysis_df["Duration (s)"] = final_df["duration_seconds"]

# User ID
analysis_df["User ID"] = final_df["submitted_by"]

# User Name
analysis_df["User Name"] = final_df["submitted_by"].apply(extract_user_name)
# Fallback to User ID if extraction failed or was null
analysis_df["User Name"] = analysis_df["User Name"].fillna(analysis_df["User ID"])

# Error Details
analysis_df["Error Message"] = coalesce_series(
    final_df["failure_reason"], 
    final_df["error_message"], 
    final_df["error_code"]
).fillna("Unknown Error")

analysis_df["Error Code"] = final_df["error_code"]

print(" Analysis DataFrame Prepared.")
print(analysis_df.head(5))

In [ ]:
# 4. Execute Analysis (Pandas)

if not analysis_df.empty:
    # --- 1. Summary Statistics ---
    total_failures = len(analysis_df)
    unique_workspaces = analysis_df["Workspace"].nunique()
    unique_items = analysis_df["Item Name"].nunique()
    
    print(f"\n SUMMARY STATISTICS")
    print(f"Total Failures: {total_failures}")
    print(f"Affected Workspaces: {unique_workspaces}")
    print(f"Affected Items: {unique_items}")

    # --- 2. Top 10 Failing Items ---
    print("\n TOP 10 FAILING ITEMS")
    top_items = analysis_df.groupby(["Workspace", "Item Name", "Item Type"]) \
        .size() \
        .reset_index(name="count") \
        .sort_values("count", ascending=False) \
        .head(10)
    print(top_items.to_string(index=False))

    # --- 3. Failures by User ---
    print("\n FAILURES BY USER")
    user_stats = analysis_df.groupby("User Name") \
        .size() \
        .reset_index(name="count") \
        .sort_values("count", ascending=False)
    print(user_stats.to_string(index=False))

    # --- 4. Error Message Distribution ---
    print("\n ERROR MESSAGE DISTRIBUTION")
    error_stats = analysis_df.groupby("Error Message") \
        .size() \
        .reset_index(name="count") \
        .sort_values("count", ascending=False)
    print(error_stats.to_string(index=False))

    # --- 5. Recent Failures (Last 20) ---
    print("\n MOST RECENT FAILURES")
    recent_failures = analysis_df[["Start Time", "Workspace", "Item Name", "User Name", "Error Message"]] \
        .sort_values("Start Time", ascending=False) \
        .head(20)
    
    # Truncate long error messages for display
    pd.set_option('display.max_colwidth', 100)
    print(recent_failures.to_string(index=False))
else:
    print("No failure data found.")

In [ ]:
analysis_df.head(5)

In [ ]:
# 5. Investigate Missing End Times and Duration Issues

print(" INVESTIGATING DURATION AND END TIME ISSUES")
print("=" * 50)

# Check the original raw data structure
print(" SAMPLE RAW ACTIVITY STRUCTURE:")
if activities:
    sample_activity = activities[0]
    for key, value in sample_activity.items():
        print(f"  {key}: {value}")

print(f"\n RAW ACTIVITIES DATA ANALYSIS:")
print(f"Total raw activities: {len(activities)}")

# Convert to DataFrame for analysis
raw_df = pd.DataFrame(activities)

# Check end_time availability in raw data
if 'end_time' in raw_df.columns:
    end_time_missing = raw_df['end_time'].isna().sum()
    end_time_total = len(raw_df)
    print(f"Missing end_time in raw data: {end_time_missing}/{end_time_total} ({end_time_missing/end_time_total*100:.1f}%)")
else:
    print(" 'end_time' column not found in raw activities")
    print("Available columns:", list(raw_df.columns))

# Check detailed jobs data for duration info
print(f"\n DETAILED JOBS DATA ANALYSIS:")
jobs_df = pd.DataFrame(detailed_jobs)
print(f"Total detailed jobs: {len(detailed_jobs)}")

if detailed_jobs:
    sample_job = detailed_jobs[0]
    print("Sample job structure:")
    for key, value in sample_job.items():
        print(f"  {key}: {value}")

# Check for duration-related fields in jobs
duration_fields = ['duration', 'durationSeconds', 'endTime', 'endTimeUtc', 'startTime', 'startTimeUtc']
available_duration_fields = [field for field in duration_fields if field in jobs_df.columns]
print(f"\nAvailable duration-related fields in jobs: {available_duration_fields}")

for field in available_duration_fields:
    if field in jobs_df.columns:
        missing_count = jobs_df[field].isna().sum()
        total_count = len(jobs_df)
        print(f"  {field}: {missing_count}/{total_count} missing ({missing_count/total_count*100:.1f}%)")

In [ ]:
# 6. Fix Duration Calculation Using Job Data

print(" IMPLEMENTING DURATION FIX")
print("=" * 40)

# Create a copy of merged_df to avoid modifying the original
fixed_df = merged_df.copy()

# Convert job times to datetime if they aren't already
if 'job_start_time' in fixed_df.columns:
    fixed_df['job_start_time'] = pd.to_datetime(fixed_df['job_start_time'], utc=True, errors='coerce')

if 'endTimeUtc' in fixed_df.columns:
    fixed_df['job_end_time'] = pd.to_datetime(fixed_df['endTimeUtc'], utc=True, errors='coerce')
else:
    # Create job_end_time from endTimeUtc if it exists in the merge
    job_columns = [col for col in fixed_df.columns if 'endTime' in col]
    print(f"Available end time columns: {job_columns}")
    
    if job_columns:
        end_time_col = job_columns[0]  # Use the first available end time column
        fixed_df['job_end_time'] = pd.to_datetime(fixed_df[end_time_col], utc=True, errors='coerce')

# Fix end_time: Use job end time when activity end time is missing
print("Fixing end_time...")
original_missing_end_time = fixed_df['end_time'].isna().sum()
print(f"  Activities missing end_time: {original_missing_end_time}")

if 'job_end_time' in fixed_df.columns:
    # Fill missing end_time with job_end_time
    fixed_df['end_time'] = fixed_df['end_time'].fillna(fixed_df['job_end_time'])
    
    after_fix_missing_end_time = fixed_df['end_time'].isna().sum()
    fixed_count = original_missing_end_time - after_fix_missing_end_time
    print(f"  Fixed {fixed_count} missing end times using job data")
    print(f"  Remaining missing end_time: {after_fix_missing_end_time}")

# Recalculate duration_seconds
print("Recalculating duration...")
def calculate_duration(start_time, end_time):
    if pd.isna(start_time) or pd.isna(end_time):
        return 0.0
    try:
        duration = (end_time - start_time).total_seconds()
        return max(0.0, duration)  # Ensure non-negative duration
    except:
        return 0.0

fixed_df['duration_seconds'] = fixed_df.apply(
    lambda row: calculate_duration(row['start_time'], row['end_time']), 
    axis=1
)

# Update the merged_activities list with fixed data
fixed_activities = fixed_df.to_dict(orient="records")

# Show improvement statistics
original_zero_duration = (merged_df['duration_seconds'] == 0.0).sum()
fixed_zero_duration = (fixed_df['duration_seconds'] == 0.0).sum()
improvement = original_zero_duration - fixed_zero_duration

print(f"\n IMPROVEMENT STATISTICS:")
print(f"  Original zero duration records: {original_zero_duration}")
print(f"  Fixed zero duration records: {fixed_zero_duration}")  
print(f"  Records with duration restored: {improvement}")

# Update the global variables for downstream analysis
merged_activities = fixed_activities
merged_df = fixed_df

print(" Duration fix applied successfully!")

In [ ]:
# 7. Regenerate Analysis with Fixed Duration Data

print(" REGENERATING ANALYSIS WITH FIXED DURATION DATA")
print("=" * 50)

# Re-prepare DataFrame for Analysis with fixed data
df_pd_fixed = pd.DataFrame(merged_activities)

# Ensure critical columns exist
for c in expected_cols:
    if c not in df_pd_fixed.columns:
        df_pd_fixed[c] = None

# Filter for Failures
final_df_fixed = df_pd_fixed[df_pd_fixed["status"] == "Failed"].copy()

# Regenerate Analysis DataFrame with fixed duration
analysis_df_fixed = pd.DataFrame()

# Workspace  
analysis_df_fixed["Workspace"] = coalesce_series(
    final_df_fixed["workspace_name"], 
    final_df_fixed["workspace_id"]
).fillna("Unknown")

# Item Name
analysis_df_fixed["Item Name"] = final_df_fixed["item_name"].fillna("Unknown")

# Item Type
analysis_df_fixed["Item Type"] = final_df_fixed["item_type"].fillna("Unknown")

# Invoke Type
analysis_df_fixed["Invoke Type"] = final_df_fixed["activity_type"]

# Time & Duration (FIXED)
analysis_df_fixed["Start Time"] = final_df_fixed["start_time"]
analysis_df_fixed["End Time"] = final_df_fixed["end_time"]
analysis_df_fixed["Duration (s)"] = final_df_fixed["duration_seconds"]

# User ID
analysis_df_fixed["User ID"] = final_df_fixed["submitted_by"]

# User Name
analysis_df_fixed["User Name"] = final_df_fixed["submitted_by"].apply(extract_user_name)
analysis_df_fixed["User Name"] = analysis_df_fixed["User Name"].fillna(analysis_df_fixed["User ID"])

# Error Details
analysis_df_fixed["Error Message"] = coalesce_series(
    final_df_fixed["failure_reason"], 
    final_df_fixed["error_message"], 
    final_df_fixed["error_code"]
).fillna("Unknown Error")

analysis_df_fixed["Error Code"] = final_df_fixed["error_code"]

# Show duration improvement
zero_duration_original = (analysis_df["Duration (s)"] == 0.0).sum()
zero_duration_fixed = (analysis_df_fixed["Duration (s)"] == 0.0).sum()
non_zero_duration_fixed = (analysis_df_fixed["Duration (s)"] > 0.0).sum()

print(f" DURATION ANALYSIS IMPROVEMENT:")
print(f"  Original analysis - Zero duration failures: {zero_duration_original}")
print(f"  Fixed analysis - Zero duration failures: {zero_duration_fixed}")
print(f"  Fixed analysis - Non-zero duration failures: {non_zero_duration_fixed}")
print(f"  Improvement: {zero_duration_original - zero_duration_fixed} failures now have duration data")

# Show sample of fixed data
print(f"\n SAMPLE OF FIXED ANALYSIS DATA:")
print(analysis_df_fixed[analysis_df_fixed["Duration (s)"] > 0].head(5))

In [ ]:
# 8. Enhanced Duration-Based Analysis

print(" ENHANCED DURATION-BASED ANALYSIS")
print("=" * 45)

# Filter for failures with valid durations
valid_duration_failures = analysis_df_fixed[analysis_df_fixed["Duration (s)"] > 0].copy()

if len(valid_duration_failures) > 0:
    print(f" DURATION STATISTICS:")
    print(f"  Failures with duration data: {len(valid_duration_failures)}")
    print(f"  Average failure duration: {valid_duration_failures['Duration (s)'].mean():.2f} seconds")
    print(f"  Median failure duration: {valid_duration_failures['Duration (s)'].median():.2f} seconds") 
    print(f"  Max failure duration: {valid_duration_failures['Duration (s)'].max():.2f} seconds")
    print(f"  Min failure duration: {valid_duration_failures['Duration (s)'].min():.2f} seconds")
    
    # Duration percentiles
    percentiles = [25, 50, 75, 90, 95, 99]
    print(f"\n  Duration Percentiles:")
    for p in percentiles:
        value = valid_duration_failures['Duration (s)'].quantile(p/100)
        print(f"    {p}th percentile: {value:.2f}s")
    
    # Longest running failures
    print(f"\n TOP 10 LONGEST RUNNING FAILURES:")
    longest_failures = valid_duration_failures.nlargest(10, "Duration (s)")
    for idx, row in longest_failures.iterrows():
        print(f"  {row['Duration (s)']:.1f}s - {row['Workspace']} / {row['Item Name']} ({row['Item Type']})")
    
    # Quick vs Long failures
    quick_threshold = 30  # 30 seconds
    long_threshold = 300  # 5 minutes
    
    quick_failures = len(valid_duration_failures[valid_duration_failures["Duration (s)"] <= quick_threshold])
    long_failures = len(valid_duration_failures[valid_duration_failures["Duration (s)"] >= long_threshold])
    
    print(f"\n FAILURE CATEGORIES BY DURATION:")
    print(f"  Quick failures (≤{quick_threshold}s): {quick_failures}")
    print(f"  Long failures (≥{long_threshold}s): {long_failures}")
    print(f"  Medium failures: {len(valid_duration_failures) - quick_failures - long_failures}")
    
else:
    print(" No failures with valid duration data found")

print(f"\n Duration analysis complete!")

In [ ]:
# Display the fixed analysis DataFrame
analysis_df_fixed.head(10)

In [ ]:
# 9. Advanced Visualizations - Install Required Libraries
import subprocess
import sys

# Install visualization libraries if not available
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import networkx as nx
    from sklearn.preprocessing import LabelEncoder
    print(" All visualization libraries already available")
except ImportError as e:
    print(f"Installing missing visualization libraries...")
    missing_libs = []
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        missing_libs.append('matplotlib')
    try:
        import seaborn as sns
    except ImportError:
        missing_libs.append('seaborn')
    try:
        import plotly.express as px
    except ImportError:
        missing_libs.append('plotly')
    try:
        import networkx as nx
    except ImportError:
        missing_libs.append('networkx')
    try:
        from sklearn.preprocessing import LabelEncoder
    except ImportError:
        missing_libs.append('scikit-learn')
    
    if missing_libs:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + missing_libs)
        print(f" Installed: {', '.join(missing_libs)}")

# Import after installation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

print(" Visualization environment ready!")

In [ ]:
# 10. Failure Timeline Analysis with Interactive Charts

print(" CREATING FAILURE TIMELINE ANALYSIS")
print("=" * 45)

# Prepare data for timeline analysis
timeline_df = analysis_df_fixed.copy()
timeline_df['Start Time'] = pd.to_datetime(timeline_df['Start Time'])
timeline_df['Date'] = timeline_df['Start Time'].dt.date
timeline_df['Hour'] = timeline_df['Start Time'].dt.hour

# Daily failure trends
daily_failures = timeline_df.groupby('Date').size().reset_index(name='Failure Count')
daily_failures['Date'] = pd.to_datetime(daily_failures['Date'])

# Interactive timeline using Plotly
fig_timeline = px.line(daily_failures, x='Date', y='Failure Count',
                      title=' Daily Failure Trends',
                      labels={'Failure Count': 'Number of Failures', 'Date': 'Date'})
fig_timeline.update_traces(line_color='red', line_width=3)
fig_timeline.update_layout(
    template='plotly_white',
    hovermode='x unified',
    showlegend=False
)
fig_timeline.show()

# Hourly heatmap
hourly_daily = timeline_df.groupby(['Date', 'Hour']).size().reset_index(name='Failures')
hourly_pivot = hourly_daily.pivot(index='Date', columns='Hour', values='Failures').fillna(0)

# Create heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(hourly_pivot, cmap='Reds', cbar_kws={'label': 'Number of Failures'})
plt.title(' Failure Heatmap: Date vs Hour of Day', fontsize=16, pad=20)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Date', fontsize=12)
plt.xticks(range(0, 24, 2))
plt.tight_layout()
plt.show()

print(f" Timeline analysis complete - {len(daily_failures)} days analyzed")

In [ ]:
# 11. Duration Distribution Analysis with Advanced Charts

print(" CREATING DURATION DISTRIBUTION ANALYSIS")
print("=" * 48)

# Filter for valid durations
duration_data = analysis_df_fixed[analysis_df_fixed["Duration (s)"] > 0].copy()

if len(duration_data) > 0:
    # Duration histogram with multiple views
    fig_duration = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Duration Histogram', 'Duration Box Plot', 
                       'Log Scale Duration', 'Duration by Item Type'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Histogram
    fig_duration.add_trace(
        go.Histogram(x=duration_data['Duration (s)'], name='Duration', 
                    nbinsx=50, marker_color='skyblue'),
        row=1, col=1
    )
    
    # Box plot
    fig_duration.add_trace(
        go.Box(y=duration_data['Duration (s)'], name='Duration Distribution',
               marker_color='lightcoral'),
        row=1, col=2
    )
    
    # Log scale histogram
    fig_duration.add_trace(
        go.Histogram(x=duration_data['Duration (s)'], name='Log Duration', 
                    nbinsx=50, marker_color='lightgreen'),
        row=2, col=1
    )
    fig_duration.update_xaxes(type="log", row=2, col=1)
    
    # Duration by Item Type
    item_type_duration = duration_data.groupby('Item Type')['Duration (s)'].mean().sort_values(ascending=True)
    fig_duration.add_trace(
        go.Bar(x=item_type_duration.values, y=item_type_duration.index, 
               orientation='h', name='Avg Duration by Type', marker_color='gold'),
        row=2, col=2
    )
    
    fig_duration.update_layout(height=800, title_text=" Comprehensive Duration Analysis", showlegend=False)
    fig_duration.show()
    
    # Static matplotlib version for duration distribution
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Duration histogram
    ax1.hist(duration_data['Duration (s)'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.set_xlabel('Duration (seconds)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Duration Distribution')
    ax1.grid(True, alpha=0.3)
    
    # Duration vs Item Type violin plot
    item_types = duration_data['Item Type'].unique()[:10]  # Top 10 types
    filtered_data = duration_data[duration_data['Item Type'].isin(item_types)]
    if len(filtered_data) > 0:
        sns.violinplot(data=filtered_data, x='Item Type', y='Duration (s)', ax=ax2)
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        ax2.set_title('Duration by Item Type')
    
    # Duration percentile analysis
    percentiles = range(10, 100, 10)
    percentile_values = [duration_data['Duration (s)'].quantile(p/100) for p in percentiles]
    ax3.plot(percentiles, percentile_values, marker='o', linewidth=2, color='red')
    ax3.set_xlabel('Percentile')
    ax3.set_ylabel('Duration (seconds)')
    ax3.set_title('Duration Percentiles')
    ax3.grid(True, alpha=0.3)
    
    # Duration scatter: Start Time vs Duration
    ax4.scatter(duration_data['Start Time'], duration_data['Duration (s)'], alpha=0.6, color='purple')
    ax4.set_xlabel('Start Time')
    ax4.set_ylabel('Duration (seconds)')
    ax4.set_title('Duration Over Time')
    plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f" Duration analysis complete - {len(duration_data)} failures with duration data")
else:
    print(" No duration data available for visualization")

In [ ]:
# 12. Workspace and Item Analysis with Heat Maps

print(" CREATING WORKSPACE & ITEM ANALYSIS")
print("=" * 42)

# Workspace failure matrix
workspace_item_matrix = analysis_df_fixed.groupby(['Workspace', 'Item Type']).size().reset_index(name='Failures')
workspace_pivot = workspace_item_matrix.pivot(index='Workspace', columns='Item Type', values='Failures').fillna(0)

# Interactive heatmap for workspace vs item type
fig_heatmap = px.imshow(workspace_pivot, 
                       title=' Workspace vs Item Type Failure Matrix',
                       labels=dict(x="Item Type", y="Workspace", color="Failures"),
                       aspect="auto", color_continuous_scale='Reds')
fig_heatmap.update_layout(height=max(400, len(workspace_pivot) * 40))
fig_heatmap.show()

# Top failing workspace-item combinations
print("\n TOP WORKSPACE-ITEM FAILURE COMBINATIONS:")
top_combinations = analysis_df_fixed.groupby(['Workspace', 'Item Name', 'Item Type']).size().sort_values(ascending=False).head(15)
for idx, ((workspace, item_name, item_type), count) in enumerate(top_combinations.items(), 1):
    print(f"{idx:2d}. {workspace} / {item_name} ({item_type}): {count} failures")

# Create sunburst chart for hierarchical view
if len(analysis_df_fixed) > 0:
    # Prepare data for sunburst
    sunburst_data = analysis_df_fixed.groupby(['Workspace', 'Item Type', 'Item Name']).size().reset_index(name='Failures')
    
    fig_sunburst = px.sunburst(sunburst_data, 
                              path=['Workspace', 'Item Type', 'Item Name'], 
                              values='Failures',
                              title=' Hierarchical Failure View: Workspace → Item Type → Item Name')
    fig_sunburst.show()

# User activity heatmap
user_hour_matrix = timeline_df.groupby(['User Name', 'Hour']).size().reset_index(name='Failures')
user_hour_pivot = user_hour_matrix.pivot(index='User Name', columns='Hour', values='Failures').fillna(0)

# Show only top 15 users by failure count
top_users = analysis_df_fixed['User Name'].value_counts().head(15).index
user_hour_pivot_filtered = user_hour_pivot.loc[user_hour_pivot.index.isin(top_users)]

plt.figure(figsize=(16, 8))
sns.heatmap(user_hour_pivot_filtered, cmap='Oranges', cbar_kws={'label': 'Number of Failures'})
plt.title(' User Activity Heatmap: Top Users vs Hour of Day', fontsize=16, pad=20)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('User Name', fontsize=12)
plt.xticks(range(0, 24, 2))
plt.tight_layout()
plt.show()

print(f" Workspace & item analysis complete")

In [ ]:
# 13. Error Analysis Dashboard with Interactive Visualizations

print(" CREATING ERROR ANALYSIS DASHBOARD")
print("=" * 42)

# Error message analysis
error_analysis = analysis_df_fixed['Error Message'].value_counts()

# Interactive error distribution pie chart
fig_errors = px.pie(values=error_analysis.values[:15], names=error_analysis.index[:15],
                   title=' Top 15 Error Message Distribution')
fig_errors.update_traces(textposition='inside', textinfo='percent+label')
fig_errors.show()

# Error trends over time
timeline_df['Error Message'] = timeline_df['Error Message'].fillna('Unknown Error')
error_timeline = timeline_df.groupby(['Date', 'Error Message']).size().reset_index(name='Count')

# Get top 5 errors for timeline
top_errors = analysis_df_fixed['Error Message'].value_counts().head(5).index.tolist()
error_timeline_filtered = error_timeline[error_timeline['Error Message'].isin(top_errors)]

fig_error_timeline = px.line(error_timeline_filtered, x='Date', y='Count', 
                            color='Error Message',
                            title=' Error Trends Over Time (Top 5 Errors)')
fig_error_timeline.update_layout(template='plotly_white')
fig_error_timeline.show()

# Error correlation matrix with item types
error_item_matrix = analysis_df_fixed.groupby(['Error Message', 'Item Type']).size().reset_index(name='Count')
error_item_pivot = error_item_matrix.pivot(index='Error Message', columns='Item Type', values='Count').fillna(0)

# Show top 10 errors
top_10_errors = error_analysis.head(10).index
error_item_pivot_filtered = error_item_pivot.loc[error_item_pivot.index.isin(top_10_errors)]

plt.figure(figsize=(14, 8))
sns.heatmap(error_item_pivot_filtered, cmap='Reds', annot=True, fmt='.0f', 
            cbar_kws={'label': 'Number of Occurrences'})
plt.title(' Error Message vs Item Type Correlation Matrix', fontsize=16, pad=20)
plt.xlabel('Item Type', fontsize=12)
plt.ylabel('Error Message', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Create a comprehensive error insights table
print("\n ERROR INSIGHTS TABLE:")
error_insights = []
for error_msg in error_analysis.head(10).index:
    error_data = analysis_df_fixed[analysis_df_fixed['Error Message'] == error_msg]
    
    insights = {
        'Error': error_msg[:50] + '...' if len(error_msg) > 50 else error_msg,
        'Count': len(error_data),
        'Avg_Duration': error_data[error_data['Duration (s)'] > 0]['Duration (s)'].mean() if len(error_data[error_data['Duration (s)'] > 0]) > 0 else 0,
        'Top_Workspace': error_data['Workspace'].mode()[0] if len(error_data) > 0 else 'N/A',
        'Top_Item_Type': error_data['Item Type'].mode()[0] if len(error_data) > 0 else 'N/A',
        'Unique_Users': error_data['User Name'].nunique()
    }
    error_insights.append(insights)

error_insights_df = pd.DataFrame(error_insights)
error_insights_df['Avg_Duration'] = error_insights_df['Avg_Duration'].round(2)
print(error_insights_df.to_string(index=False))

print(f"\n Error analysis dashboard complete")

In [ ]:
# 14. Performance Insights and Predictive Analytics

print(" CREATING PERFORMANCE INSIGHTS & PREDICTIVE ANALYTICS")
print("=" * 58)

# Performance metrics calculation
perf_metrics = {}

# Calculate failure rates by various dimensions
total_records = len(analysis_df_fixed)
perf_metrics['total_failures'] = total_records

# Failure rate by workspace
workspace_failure_rate = analysis_df_fixed.groupby('Workspace').size().sort_values(ascending=False)
perf_metrics['worst_workspace'] = workspace_failure_rate.index[0] if len(workspace_failure_rate) > 0 else 'N/A'
perf_metrics['worst_workspace_failures'] = workspace_failure_rate.iloc[0] if len(workspace_failure_rate) > 0 else 0

# Failure rate by user
user_failure_rate = analysis_df_fixed.groupby('User Name').size().sort_values(ascending=False)
perf_metrics['most_errors_user'] = user_failure_rate.index[0] if len(user_failure_rate) > 0 else 'N/A'
perf_metrics['most_errors_count'] = user_failure_rate.iloc[0] if len(user_failure_rate) > 0 else 0

# Failure rate by item type
item_type_failure_rate = analysis_df_fixed.groupby('Item Type').size().sort_values(ascending=False)

# Performance dashboard
fig_perf = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Failures by Workspace', 'Failures by Item Type', 
                   'Failures by User (Top 15)', 'Daily Failure Trend'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Workspace failures
top_workspaces = workspace_failure_rate.head(10)
fig_perf.add_trace(
    go.Bar(x=top_workspaces.values, y=top_workspaces.index, orientation='h', 
           name='Workspace Failures', marker_color='lightblue'),
    row=1, col=1
)

# Item type pie chart
fig_perf.add_trace(
    go.Pie(labels=item_type_failure_rate.head(8).index, 
           values=item_type_failure_rate.head(8).values,
           name='Item Type Distribution'),
    row=1, col=2
)

# User failures
top_users_perf = user_failure_rate.head(15)
fig_perf.add_trace(
    go.Bar(x=top_users_perf.index, y=top_users_perf.values,
           name='User Failures', marker_color='lightcoral'),
    row=2, col=1
)

# Daily trend
fig_perf.add_trace(
    go.Scatter(x=daily_failures['Date'], y=daily_failures['Failure Count'],
               mode='lines+markers', name='Daily Failures', line_color='darkred'),
    row=2, col=2
)

fig_perf.update_layout(height=900, title_text=" Performance Dashboard Overview", showlegend=False)
fig_perf.update_xaxes(tickangle=45, row=2, col=1)
fig_perf.show()

# Predictive insights based on patterns
print("\n PREDICTIVE INSIGHTS:")
print("=" * 25)

# Peak failure times
peak_hour = timeline_df['Hour'].mode()[0] if len(timeline_df) > 0 else 'N/A'
peak_day = timeline_df['Date'].value_counts().index[0] if len(timeline_df) > 0 else 'N/A'

print(f" Peak Failure Hour: {peak_hour}:00")
print(f" Worst Failure Day: {peak_day}")

# Average failure patterns
if len(duration_data) > 0:
    avg_failure_duration = duration_data['Duration (s)'].mean()
    print(f" Average Failure Duration: {avg_failure_duration:.2f} seconds")
    
    # Duration-based risk categorization
    quick_failures = len(duration_data[duration_data['Duration (s)'] <= 30])
    medium_failures = len(duration_data[(duration_data['Duration (s)'] > 30) & (duration_data['Duration (s)'] <= 300)])
    long_failures = len(duration_data[duration_data['Duration (s)'] > 300])
    
    print(f" Quick Failures (≤30s): {quick_failures} ({quick_failures/len(duration_data)*100:.1f}%)")
    print(f" Medium Failures (30s-5m): {medium_failures} ({medium_failures/len(duration_data)*100:.1f}%)")  
    print(f" Long Failures (>5m): {long_failures} ({long_failures/len(duration_data)*100:.1f}%)")

# Risk assessment
print(f"\n RISK ASSESSMENT:")
print("=" * 20)

risk_factors = []
if perf_metrics['worst_workspace_failures'] > total_records * 0.3:
    risk_factors.append(f"High workspace concentration: {perf_metrics['worst_workspace']} has {perf_metrics['worst_workspace_failures']} failures")

if perf_metrics['most_errors_count'] > total_records * 0.2:
    risk_factors.append(f"High user concentration: {perf_metrics['most_errors_user']} has {perf_metrics['most_errors_count']} failures")

if len(risk_factors) > 0:
    for i, factor in enumerate(risk_factors, 1):
        print(f"{i}. {factor}")
else:
    print(" No major risk concentrations detected")

print(f"\n Performance insights analysis complete")

In [ ]:
# 15. Advanced Correlation Analysis and Network Visualization

print(" CREATING CORRELATION & NETWORK ANALYSIS")
print("=" * 46)

# Create correlation analysis between different failure dimensions
correlation_data = analysis_df_fixed.copy()

# Install sklearn if not available and import
try:
    from sklearn.preprocessing import LabelEncoder
except ImportError:
    print("Installing scikit-learn for correlation analysis...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    from sklearn.preprocessing import LabelEncoder
    print(" scikit-learn installed successfully")

categorical_cols = ['Workspace', 'Item Name', 'Item Type', 'User Name', 'Error Message']
encoded_data = correlation_data.copy()

# Encode categorical variables for correlation analysis
print(" Encoding categorical variables for correlation analysis...")
for col in categorical_cols:
    if col in encoded_data.columns:
        le = LabelEncoder()
        # Handle missing values by converting to string first
        encoded_data[f'{col}_encoded'] = le.fit_transform(encoded_data[col].astype(str).fillna('Unknown'))

# Calculate correlations between numeric columns
numeric_cols = ['Duration (s)'] + [f'{col}_encoded' for col in categorical_cols if f'{col}_encoded' in encoded_data.columns]
correlation_matrix = encoded_data[numeric_cols].corr()

# Correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title(' Failure Factor Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Network analysis - connection between workspaces and item types
print("\n Creating network visualization...")

# Install networkx if not available
try:
    import networkx as nx
except ImportError:
    print("Installing networkx for network analysis...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'networkx'])
    import networkx as nx
    print(" networkx installed successfully")

# Create network graph
G = nx.Graph()

# Add edges between workspaces and item types weighted by failure count
workspace_item_connections = analysis_df_fixed.groupby(['Workspace', 'Item Type']).size().reset_index(name='weight')

for _, row in workspace_item_connections.iterrows():
    if row['weight'] > 1:  # Only include connections with more than 1 failure
        G.add_edge(f"WS: {row['Workspace']}", f"IT: {row['Item Type']}", weight=row['weight'])

# Create network visualization
if len(G.nodes()) > 0:
    plt.figure(figsize=(15, 12))
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)

    # Draw nodes
    workspace_nodes = [n for n in G.nodes() if n.startswith('WS:')]
    itemtype_nodes = [n for n in G.nodes() if n.startswith('IT:')]

    nx.draw_networkx_nodes(G, pos, nodelist=workspace_nodes, node_color='lightblue', 
                          node_size=1000, alpha=0.8, label='Workspaces')
    nx.draw_networkx_nodes(G, pos, nodelist=itemtype_nodes, node_color='lightcoral', 
                          node_size=800, alpha=0.8, label='Item Types')

    # Draw edges with thickness based on weight
    edges = G.edges()
    weights = [G[u][v]['weight'] for u, v in edges]
    max_weight = max(weights) if weights else 1
    normalized_weights = [max(1, w/max_weight * 5) for w in weights]  # Normalize for better visualization
    
    nx.draw_networkx_edges(G, pos, width=normalized_weights, alpha=0.6, edge_color='gray')

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')

    plt.title(' Workspace-ItemType Failure Network', fontsize=16, pad=20)
    plt.legend()
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print(" No network connections found (need >1 failure per workspace-itemtype pair)")

# Co-occurrence analysis - which errors tend to happen together
print("\n ERROR CO-OCCURRENCE ANALYSIS:")
print("=" * 35)

# Group by user and time window to find errors that occur together
correlation_data['Time_Window'] = correlation_data['Start Time'].dt.floor('H')  # Hour windows
error_cooccurrence = correlation_data.groupby(['User Name', 'Time_Window'])['Error Message'].apply(list).reset_index()

# Find pairs of errors that occur together frequently
from collections import Counter
from itertools import combinations

error_pairs = []
for error_list in error_cooccurrence['Error Message']:
    if len(error_list) > 1:
        pairs = list(combinations(set(error_list), 2))  # Get unique pairs
        error_pairs.extend(pairs)

if error_pairs:
    pair_counts = Counter(error_pairs)
    top_pairs = pair_counts.most_common(10)

    print("Top 10 Error Co-occurrences (same user, same hour):")
    for i, ((error1, error2), count) in enumerate(top_pairs, 1):
        error1_short = error1[:30] + '...' if len(error1) > 30 else error1
        error2_short = error2[:30] + '...' if len(error2) > 30 else error2
        print(f"{i:2d}. {error1_short}  {error2_short} ({count} times)")
else:
    print("ℹ No error co-occurrences found (errors don't typically happen together)")

print(f"\n Correlation & network analysis complete")

In [ ]:
# 16. Executive Summary Dashboard and Key Metrics

print(" CREATING EXECUTIVE SUMMARY DASHBOARD")
print("=" * 45)

# Calculate key executive metrics
exec_metrics = {}

# Overall health metrics
total_failures = len(analysis_df_fixed)
unique_workspaces = analysis_df_fixed['Workspace'].nunique()
unique_users = analysis_df_fixed['User Name'].nunique()
unique_items = analysis_df_fixed['Item Name'].nunique()
date_range = (timeline_df['Date'].max() - timeline_df['Date'].min()).days + 1

exec_metrics.update({
    'total_failures': total_failures,
    'unique_workspaces': unique_workspaces,
    'unique_users': unique_users,
    'unique_items': unique_items,
    'analysis_period_days': date_range,
    'daily_avg_failures': total_failures / date_range if date_range > 0 else 0
})

# Duration metrics (if available)
duration_available = len(analysis_df_fixed[analysis_df_fixed['Duration (s)'] > 0])
exec_metrics['duration_coverage'] = (duration_available / total_failures * 100) if total_failures > 0 else 0

if duration_available > 0:
    duration_stats = analysis_df_fixed[analysis_df_fixed['Duration (s)'] > 0]['Duration (s)']
    exec_metrics.update({
        'avg_failure_duration': duration_stats.mean(),
        'max_failure_duration': duration_stats.max(),
        'median_failure_duration': duration_stats.median()
    })

# Create executive dashboard
fig_exec = make_subplots(
    rows=3, cols=2,
    subplot_titles=(' Failure Volume Overview', ' Duration Analysis', 
                   ' Top Affected Workspaces', ' User Impact Distribution',
                   ' Daily Failure Trend', ' Key Performance Indicators'),
    specs=[[{"type": "indicator"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "pie"}], 
           [{"type": "scatter"}, {"type": "table"}]]
)

# KPI Indicators
fig_exec.add_trace(
    go.Indicator(
        mode="number+gauge+delta",
        value=total_failures,
        title={"text": "Total Failures"},
        gauge={'axis': {'range': [None, total_failures * 1.5]},
               'bar': {'color': "darkred"},
               'steps': [{'range': [0, total_failures * 0.5], 'color': "lightgray"},
                        {'range': [total_failures * 0.5, total_failures], 'color': "gray"}],
               'threshold': {'line': {'color': "red", 'width': 4},
                           'thickness': 0.75, 'value': total_failures}}
    ), row=1, col=1
)

# Duration distribution
if duration_available > 0:
    duration_bins = ['<30s', '30s-5m', '5m-1h', '>1h']
    duration_counts = [
        len(duration_stats[duration_stats <= 30]),
        len(duration_stats[(duration_stats > 30) & (duration_stats <= 300)]),
        len(duration_stats[(duration_stats > 300) & (duration_stats <= 3600)]),
        len(duration_stats[duration_stats > 3600])
    ]
    
    fig_exec.add_trace(
        go.Bar(x=duration_bins, y=duration_counts, name='Duration Distribution',
               marker_color=['green', 'yellow', 'orange', 'red']),
        row=1, col=2
    )

# Top workspaces
top_ws = analysis_df_fixed['Workspace'].value_counts().head(8)
fig_exec.add_trace(
    go.Bar(x=top_ws.index, y=top_ws.values, name='Workspace Failures',
           marker_color='lightblue'),
    row=2, col=1
)

# User impact pie chart
user_impact_bins = ['1 failure', '2-5 failures', '6-10 failures', '>10 failures']
user_failures = analysis_df_fixed['User Name'].value_counts()
user_impact_counts = [
    len(user_failures[user_failures == 1]),
    len(user_failures[(user_failures >= 2) & (user_failures <= 5)]),
    len(user_failures[(user_failures >= 6) & (user_failures <= 10)]),
    len(user_failures[user_failures > 10])
]

fig_exec.add_trace(
    go.Pie(labels=user_impact_bins, values=user_impact_counts, name='User Impact'),
    row=2, col=2
)

# Daily trend
fig_exec.add_trace(
    go.Scatter(x=daily_failures['Date'], y=daily_failures['Failure Count'],
               mode='lines+markers', name='Daily Failures', 
               line=dict(color='red', width=3)),
    row=3, col=1
)

# KPI Table
kpi_data = [
    ['Total Failures', f"{exec_metrics['total_failures']:,}"],
    ['Analysis Period', f"{exec_metrics['analysis_period_days']} days"],
    ['Daily Average', f"{exec_metrics['daily_avg_failures']:.1f}"],
    ['Affected Workspaces', f"{exec_metrics['unique_workspaces']}"],
    ['Affected Users', f"{exec_metrics['unique_users']}"], 
    ['Affected Items', f"{exec_metrics['unique_items']}"],
    ['Duration Coverage', f"{exec_metrics['duration_coverage']:.1f}%"]
]

if 'avg_failure_duration' in exec_metrics:
    kpi_data.extend([
        ['Avg Duration', f"{exec_metrics['avg_failure_duration']:.1f}s"],
        ['Max Duration', f"{exec_metrics['max_failure_duration']:.1f}s"]
    ])

fig_exec.add_trace(
    go.Table(
        header=dict(values=['Metric', 'Value'], fill_color='lightblue'),
        cells=dict(values=list(zip(*kpi_data)), fill_color='white')
    ), row=3, col=2
)

fig_exec.update_layout(height=1200, title_text=" Executive Summary Dashboard", showlegend=False)
fig_exec.update_xaxes(tickangle=45, row=2, col=1)
fig_exec.show()

# Summary insights
print(f"\n EXECUTIVE SUMMARY:")
print("=" * 22)
print(f" Analysis Period: {exec_metrics['analysis_period_days']} days")
print(f" Total Failures: {exec_metrics['total_failures']:,}")
print(f" Daily Average: {exec_metrics['daily_avg_failures']:.1f} failures/day")
print(f" Workspaces Affected: {exec_metrics['unique_workspaces']}")
print(f" Users Affected: {exec_metrics['unique_users']}")
print(f" Items Affected: {exec_metrics['unique_items']}")

if 'avg_failure_duration' in exec_metrics:
    print(f" Average Failure Duration: {exec_metrics['avg_failure_duration']:.1f} seconds")
    print(f" Duration Data Coverage: {exec_metrics['duration_coverage']:.1f}%")

print(f"\n Executive summary dashboard complete")

In [ ]:
# 17. Export Analysis Results and Generate Reports

print(" CREATING EXPORT & REPORTING FUNCTIONALITY")
print("=" * 48)

import json
from datetime import datetime

# Create comprehensive results dictionary for export
export_results = {
    'analysis_metadata': {
        'generated_at': datetime.now().isoformat(),
        'total_failures_analyzed': len(analysis_df_fixed),
        'analysis_period_days': date_range,
        'duration_data_coverage_pct': exec_metrics['duration_coverage'],
        'notebook_version': 'Monitor_Hub_Analysis_Fix_v1.0'
    },
    
    'summary_statistics': {
        'total_failures': exec_metrics['total_failures'],
        'unique_workspaces': exec_metrics['unique_workspaces'],
        'unique_users': exec_metrics['unique_users'],
        'unique_items': exec_metrics['unique_items'],
        'daily_average_failures': exec_metrics['daily_avg_failures']
    },
    
    'top_insights': {
        'worst_workspace': workspace_failure_rate.index[0] if len(workspace_failure_rate) > 0 else 'N/A',
        'worst_workspace_failures': int(workspace_failure_rate.iloc[0]) if len(workspace_failure_rate) > 0 else 0,
        'most_affected_user': user_failure_rate.index[0] if len(user_failure_rate) > 0 else 'N/A',
        'most_affected_user_failures': int(user_failure_rate.iloc[0]) if len(user_failure_rate) > 0 else 0,
        'most_common_error': error_analysis.index[0] if len(error_analysis) > 0 else 'N/A',
        'most_common_error_count': int(error_analysis.iloc[0]) if len(error_analysis) > 0 else 0,
        'peak_failure_hour': int(peak_hour) if peak_hour != 'N/A' else None,
        'worst_failure_day': str(peak_day) if peak_day != 'N/A' else None
    }
}

# Add duration statistics if available
if 'avg_failure_duration' in exec_metrics:
    export_results['duration_statistics'] = {
        'average_duration_seconds': exec_metrics['avg_failure_duration'],
        'median_duration_seconds': exec_metrics['median_failure_duration'],
        'max_duration_seconds': exec_metrics['max_failure_duration'],
        'records_with_duration': duration_available,
        'quick_failures_under_30s': quick_failures if 'quick_failures' in locals() else 0,
        'medium_failures_30s_to_5m': medium_failures if 'medium_failures' in locals() else 0,
        'long_failures_over_5m': long_failures if 'long_failures' in locals() else 0
    }

# Generate detailed tables for export
print(" GENERATING DETAILED EXPORT TABLES:")

# Top 20 failing items
top_failing_items = analysis_df_fixed.groupby(['Workspace', 'Item Name', 'Item Type']).size().sort_values(ascending=False).head(20)
export_results['top_failing_items'] = [
    {
        'workspace': workspace,
        'item_name': item_name, 
        'item_type': item_type,
        'failure_count': int(count)
    }
    for (workspace, item_name, item_type), count in top_failing_items.items()
]

# Top 20 error messages
top_errors_export = error_analysis.head(20)
export_results['top_error_messages'] = [
    {
        'error_message': error_msg,
        'occurrence_count': int(count),
        'percentage_of_total': round((count / len(analysis_df_fixed)) * 100, 2)
    }
    for error_msg, count in top_errors_export.items()
]

# User impact analysis
user_impact_export = user_failure_rate.head(20)
export_results['top_affected_users'] = [
    {
        'user_name': user,
        'failure_count': int(count),
        'percentage_of_total': round((count / len(analysis_df_fixed)) * 100, 2)
    }
    for user, count in user_impact_export.items()
]

# Daily failure trends
daily_trends_export = daily_failures.copy()
daily_trends_export['Date'] = daily_trends_export['Date'].astype(str)
export_results['daily_failure_trends'] = daily_trends_export.to_dict('records')

# Save results to JSON file
output_file = f"{OUTPUT_DIR}/failure_analysis_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
try:
    # Ensure output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    with open(output_file, 'w') as f:
        json.dump(export_results, f, indent=2, default=str)
    print(f" Analysis results exported to: {output_file}")
except Exception as e:
    print(f" Could not save to file system: {e}")
    print(" Results available in memory as 'export_results' variable")

# Create CSV exports for key tables
try:
    # Export main analysis data
    csv_file = f"{OUTPUT_DIR}/failure_analysis_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    analysis_df_fixed.to_csv(csv_file, index=False)
    print(f" Detailed failure data exported to: {csv_file}")
    
    # Export summary tables
    summary_file = f"{OUTPUT_DIR}/failure_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    summary_data = []
    summary_data.append(['Metric', 'Value'])
    for key, value in export_results['summary_statistics'].items():
        summary_data.append([key.replace('_', ' ').title(), value])
    
    summary_df = pd.DataFrame(summary_data[1:], columns=summary_data[0])
    summary_df.to_csv(summary_file, index=False)
    print(f" Summary statistics exported to: {summary_file}")
    
except Exception as e:
    print(f" Could not save CSV files: {e}")

# Display final summary
print(f"\n ANALYSIS COMPLETE - COMPREHENSIVE REPORT GENERATED")
print("=" * 54)
print(f" Total Visualizations Created: 16+ charts and graphs")
print(f" Analysis Types: Timeline, Heatmaps, Distributions, Correlations, Networks")
print(f" Insights Generated: Performance, Predictive, Executive Summary")
print(f" Export Formats: JSON, CSV, In-Memory DataFrames") 
print(f" Smart Merge Technology: Utilized throughout all analyses")

# Show what's available for further analysis
print(f"\n AVAILABLE DATA OBJECTS FOR FURTHER ANALYSIS:")
print(f"  - analysis_df_fixed: Main analysis DataFrame ({len(analysis_df_fixed)} records)")
print(f"  - export_results: Complete analysis results dictionary")
print(f"  - duration_data: Failures with duration information ({len(duration_data)} records)")
print(f"  - timeline_df: Time-based analysis data")
print(f"  - daily_failures: Daily aggregated failure counts")

print(f"\n Export and reporting functionality complete!")